# Example 1: Basic Workflow

This example shows the simplest way to process frame overlap data using the Workflow class.

In [ ]:
import matplotlib.pyplot as plt
from frame_overlap import Workflow

%matplotlib inline

## Complete Pipeline in One Chain

Process data from loading through analysis in a single fluent expression.

In [ ]:
# Initialize workflow with original measurement parameters
wf = Workflow(
    signal_path='iron_powder.csv',
    openbeam_path='openbeam.csv',
    flux=5e6,      # Original flux: 5×10⁶ n/cm²/s
    duration=0.5,  # Original duration: 0.5 hours
    freq=20        # Original frequency: 20 Hz
)

# Chain the complete pipeline
result = (wf
    .convolute(pulse_duration=200)                        # Apply 200 µs pulse
    .poisson(flux=1e6, freq=60, measurement_time=30)     # Scale to new conditions
    .overlap(kernel=[0, 25])                              # 2-frame overlap
    .reconstruct(kind='wiener', noise_power=0.01)        # Wiener deconvolution
    .analyze(xs='iron', vary_background=True,            # Fit with nbragg
             vary_response=True))

print(f"Workflow complete: {wf}")
print(f"\nFinal χ²/dof: {wf.result.redchi:.2f}")

## Visualize Results

In [ ]:
# Plot the final analysis
wf.plot()
plt.show()

## Access Results

The workflow stores all intermediate and final results.

In [ ]:
# Access different stages
print("Data object:", wf.data)
print("Reconstruction:", wf.recon)
print("Analysis:", wf.analysis)

# Access fitted parameters
print("\nFitted Parameters:")
for name, param in wf.result.params.items():
    if param.vary:
        print(f"  {name}: {param.value:.4f} ± {param.stderr:.4f}")

## Plot Intermediate Stages

In [ ]:
# Plot reconstruction quality
wf.recon.plot(kind='transmission', show_errors=False, ylim=(0, 1))
plt.show()

# Plot data stages
wf.data.plot(kind='signal', show_stages=True, show_errors=False, ylim=(0, 5e3))
plt.show()

## Key Takeaways

- **One chain**: Complete pipeline in a single fluent expression
- **Automatic flux scaling**: Poisson automatically uses pulse_duration from convolute
- **All results accessible**: Data, reconstruction, and analysis stored in workflow
- **Easy visualization**: `.plot()` shows current stage results